In [3]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
!pip install --upgrade mxnet-cu100
!pip install autogluon
!pip install category_encoders
!pip install -U ipykernel

--2020-05-22 03:26:49--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2571184 (2.5M) [text/plain]
Saving to: ‘doctor_train.csv’

doctor_train.csv    100%[===================>]   2.45M  --.-KB/s    in 0.09s   

2020-05-22 03:26:49 (26.9 MB/s) - ‘doctor_train.csv’ saved [2571184/2571184]

--2020-05-22 03:26:55--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaitin

     |████████████████████████████████| 81kB 2.3MB/s 
     |████████████████████████████████| 122kB 2.8MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 0.25.3 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [0]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
tdf = pd.read_csv('doctor_test.csv')

cols = ['age','Money','day','Time','Doctor_visits','last_visit',
        'cured_in','Profession','Status','edu','Irregular','residence',
        'prev_diagnosed','communication','Month','side_effects']
dep_var = ['Y']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

In [2]:
# df.info()
def dataessing(df, test):
  cat_col = ['Profession','Status','edu','Irregular','residence','prev_diagnosed','communication','Month','side_effects']
  num_col = ['age','day','Time','Doctor_visits','last_visit','cured_in']
  loan = ['Money']
  target = 'Y'
  train_DF = pd.DataFrame(); valid_DF = pd.DataFrame(); test_DF = pd.DataFrame();
  dfs = [df, test]
  for i in range(0,2):
    d = dfs[i]
    dfc = pd.DataFrame()
    dfc[cat_col] = d[cat_col].fillna("unknown")
    dfc[num_col+loan] = d[num_col+loan].fillna(0)
    dfc[num_col] = dfc[num_col].clip(lower=0)
    dfcd = pd.get_dummies(dfc, columns=cat_col)
    numcols = dfcd.columns
    
    if i == 0:
      from sklearn.model_selection import train_test_split
      train_DF, valid_DF, y_train, y_test = train_test_split(dfcd, df[target], random_state=45, test_size=0.3)
      train_DF['Y'] = y_train
      valid_DF['Y'] = y_test
    else:
      test_DF = dfcd
  return train_DF, valid_DF, test_DF, list(numcols), cat_col, target

train_DF, valid_DF, test_DF, num_col, cat_col, target = dataessing(df, tdf)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
# import category_encoders as ce
# cb_enc = ce.CatBoostEncoder(cols=cat_col, random_state=7)
# # Learn encoding from the training set
# cb_enc.fit(train_DF[cat_col], train_DF['Y'])
# # Apply encoding to the train and validation sets
# train_DF[cat_col] = cb_enc.transform(train_DF[cat_col])
# valid_DF[cat_col] = cb_enc.transform(valid_DF[cat_col])

In [4]:
import autogluon as ag
from autogluon import TabularPrediction as task
import time
predictor = task.fit(train_data=train_DF, 
                     label='Y', 
                     eval_metric='roc_auc',
                     auto_stack=True,
                     verbosity=2)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200522_033224/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200522_033224/
Train Data Rows:    19737
Train Data Columns: 54
Preprocessing data ...
Here are the first 10 unique label values in your data:  [0 1]
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: [('binary', 'multiclass', 'regression')])

Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Feature Generator processed 19737 data points with 53 features
Original Features:
	float features: 2
	int features: 51
	object features: 0
Generated Features:
	int features: 0
All Features:
	float features: 2
	int features: 51
	object features: 0
	Data preprocessing and feature engineering runtime = 0.19s ...
AutoGluon will gauge predictive performanc

In [5]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                                    model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer
0                 weighted_ensemble_k0_l1   0.936140      10.862378   764.251717                0.007308           6.701477            1       True
1                 weighted_ensemble_k0_l2   0.935747      23.940620  1526.987571                0.007375           6.625845            2       True
2           CatboostClassifier_STACKER_l1   0.934402      14.366230   829.933046                0.122707          23.408926            1       True
3           CatboostClassifier_STACKER_l0   0.933464       0.111270    36.288693                0.111270          36.288693            0       True
4   RandomForestClassifierEntr_STACKER_l1   0.933050      16.398988   883.958339                2.155465          77.434219            1       True
5           LightGBMClassifier_STACKER_l1   0.9330

In [6]:
%%time
# y_hat = predictor.predict(test_df_D)
y_hat = predictor.predict(valid_DF.drop('Y',1))

CPU times: user 46.2 s, sys: 3.9 s, total: 50.1 s
Wall time: 1min 35s


In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
print(accuracy_score(valid_DF['Y'], y_hat))
print(confusion_matrix(valid_DF['Y'], y_hat))
print(classification_report(valid_DF['Y'], y_hat))
print(roc_auc_score(valid_DF['Y'], y_hat))

0.9105095164913111
[[7269  222]
 [ 535  433]]
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      7491
           1       0.66      0.45      0.53       968

    accuracy                           0.91      8459
   macro avg       0.80      0.71      0.74      8459
weighted avg       0.90      0.91      0.90      8459

0.7088392434557833


In [0]:
# test_DF[cat_col] = cb_enc.transform(test_DF[cat_col])
valid_pred = predictor.predict(test_DF)

In [0]:
le_y = le.inverse_transform(valid_pred)
output = pd.DataFrame({'id': tdf.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')